In [5]:
import numpy as np
import compiledFunctions as cF
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def test_function(r, alpha): #very slow for some reason, so also implemented in fortran
    r1  = np.sqrt(np.sum(r[:,:3]**2,1))
    r2  = np.sqrt(np.sum(r[:,3:]**2,1))
    r12 = np.sqrt(np.sum((r[:,:3]-r[:,3:])*(r[:,:3]-r[:,3:]),1))
    return np.exp(-2*(r1+r2) + r12/(2*(1 + alpha*r12)))

def local_energy(r, alpha): #very slow for some reason, so also implemented in fortran
    r1  = np.sqrt(np.sum(r[:,:3]**2,1))
    r2  = np.sqrt(np.sum(r[:,3:]**2,1))
    r12 = np.sqrt(np.sum((r[:,:3]-r[:,3:])*(r[:,:3]-r[:,3:]),1))
    r12special = np.sum((r[:,:3]/r1[:,np.newaxis] - r[:,3:]/r2[:,np.newaxis])*(r[:,:3]-r[:,3:]),1)/r12
    return (-4 + r12special/(1 + alpha*r12)**2 - 1/(r12*(1 + alpha*r12)**3) - 1/(4*(1 + alpha*r12)**4) + 1/r12)
    
    
def doTheWalkHelium(alpha, N, m_iterations, d_max):
    r_old = np.random.rand(N,6) - 0.5
    r_new = r_old
    psi_old = cF.helium_test_function(r_old, alpha)
    psi_new = psi_old
    
    El = np.zeros((m_iterations, N))
    rolled_dice = np.random.rand(m_,N)
    for i in range(m_iterations):
        #### MOVE WALKERS
        dr = d_max * (np.random.rand(N,6)-0.5)
        r_new = r_old + dr
        
        #### CALC LOCAL PSI
        psi_new = cF.helium_test_function(r_new, alpha, N)
        
        #### CALC ACCEPTANCE ODDS
        p_accept = (psi_new / psi_old)**2
        
        #### 'ROLL DICE' AND ACCEPT OR REJECT NEW POSITIONS
        accepted_indices = rolled_dice[i,:] <= p_accept
        r_old[accepted_indices, :] = r_new[accepted_indices, :]
        psi_old[accepted_indices] = psi_new[accepted_indices]
        
        #### CALC LOC ENERGY
        El[i,:] = cF.helium_local_energy(r_old, alpha, N)
    return El


def find_a(s): #Hideous I know, but the lambertw functions has issues on my system
    def a_func(a,s):
        return np.abs(a*(1+np.exp(-s/a))-1)
    a_structure = minimize(a_func,0.85,s)
    if np.abs(a_func(a_structure.x,s))>0.001:
        print('ERROR IN a(s) CALCULATION')
    return a_structure.x

def doTheWalkHydrogen(beta, s, N, m_iterations, d_max):
    cutoff = 10000
    a = find_a(s)
    Elv = np.zeros(N)
    Elm = np.zeros(N)  
    
    r_old = d_max * (np.random.rand(N,6) - 0.5)
    r_new = r_old
    psi_old = cF.hydrogen_test_function(r_old, beta, s, a)
    psi_new = psi_old

    for i in range(m_iterations):
        #### MOVE WALKERS
        dr = d_max * (np.random.rand(N,6)-0.5)
        r_new = r_old + dr
        
        #### CALC LOCAL PSI
        psi_new = cF.hydrogen_test_function(r_new, beta, s, a)
        
        #### CALC ACCEPTANCE ODDS
        p_accept = (psi_new / psi_old)**2
        
        #### 'ROLL DICE' AND ACCEPT OR REJECT NEW POSITIONS
        rolled_dice = np.random.rand(N)
        accepted_indices = rolled_dice <= p_accept
        r_old[accepted_indices, :] = r_new[accepted_indices, :]
        psi_old[accepted_indices] = psi_new[accepted_indices]
        
        #### CALC LOC ENERGY
        (El_i, steep_descent_beta) = cF.hydrogen_local_energy(r_old, beta, s, a)
        if i>cutoff:
            Elm += El_i
            Elv += El_i**2
        
    Elm /= (m_iterations-cutoff)
    Elv /= (m_iterations-cutoff)
    return (Elm, Elv)


In [ ]:
v_beta = np.linspace(0.4,.55,2)
v_s = np.linspace(1.39,1.41,1)
N_walkers = 400

(M_beta,M_s) = np.meshgrid(v_beta,v_s)
M_beta = M_beta.flatten()
M_s = M_s.flatten()

Elm = np.zeros((N_walkers, M_s.size))
Elv = np.zeros((N_walkers, M_s.size))

for i in range(M_s.size):
    (Elm[:,i], Elv[:,i]) = doTheWalkHydrogen(M_beta[i],M_s[i],N_walkers,30000,1)
    
Elv_walkermean = np.mean(Elv - Elm**2,0)
Elm_walkermean = np.mean(Elm, 0)

# As walkers have tendency to get stuck, we cannot consider the datapoints in the iteration
# axis to be part of the spatial population. Instead, we set the number of iterations high 
# and (rightfully) assume their contribution to the total error to be negligible. Instead
# we assume the error to arise due to the initial placement of the N_walkers.
Elv_walkermean_error = np.var(Elv - Elm**2, 0)/ np.sqrt(N_walkers)
Elm_walkermean_error = np.var(Elm, 0)/ np.sqrt(N_walkers)

for i in range(M_s.size):
    print('s = {0:.2f},beta = {1:.2f}|\t<E>= {2:.5f} +- {3:.2e}|\tVar(<E>)= {4:.5f} +- {5:.2e}'
                        .format(M_s[i], M_beta[i],
                        Elm_walkermean[i], Elm_walkermean_error[i],
                        Elv_walkermean[i], Elv_walkermean_error[i]))